In [0]:
import numpy as np
import random
from collections import defaultdict
import sys
import time


class Point:
    def __init__(self, label, doc_id, tfidf):
        def normalize():
            # compute norm2 square
            ans = 0.0
            for x in tfidf.values():
                ans += x**2
            ans=ans**0.5
            for i in tfidf:
                tfidf[i] = tfidf[i]/ans

            self.tfidf = tfidf

        self.label = label
        self.doc_id = doc_id
        normalize()


class Cluster:
    def __init__(self):
        self.centroid = None
        self.points = []
        self.centroid_l2_square = 0.0

    def set_centroid(self, new_centroid):
        self.centroid = new_centroid

    def add_point(self, point):
        self.points.append(point)

    def reset_points(self):
        self.points = []

def Load_data(pathin):
    def tfidf(doc):
        tf_idf = defaultdict(int)
        fea = doc.split()
        for i in fea:
            tmp = i.split(':')
            tf_idf[int(tmp[0])] = float(tmp[1])
        return tf_idf

    with open(pathin, 'r') as f:
        d_lines = f.read().splitlines()
    data = []
    for d in d_lines:
        fea = d.split('<<>>')
        label = fea[0]
        doc_id = fea[1]
        data.append(Point(label, doc_id,tfidf(fea[2])))
    return data

X_train = Load_data("/content/drive/My Drive/Project2/Datasets/train_tfidf_df=3")
X_test = Load_data("/content/drive/My Drive/Project2/Datasets/test_tfidf")



In [0]:

class Kmeans:
    def __init__(self):
        self.list_clusters = []
        self.k_cluster = 0
        self.n_doc = 0
        self.new_clusters = []
        self.max_simirality = 0.0
        self.it=0
        '''
        with open("/content/drive/My Drive/Project2/Datasets/words_idf", 'r') as f:
            self.dim = len(f.read().splitlines())
        '''
        self.dim=20167

    def dist_between_x_y(self, x, y):
        '''
        x is a point;
        y is a vector
        '''
        # compute dot product
        ans = 0.0
        for pos in x.tfidf:
            ans += x.tfidf[pos]*y[pos]
        # return distance
        return (2-2*ans)

    def init_centroid(self, X):
        '''
        init centroid with K-means algorithm
        '''
        rd = random.sample(range(self.n_doc), self.k_cluster)
        for i in rd:
            centroid = np.zeros(self.dim)
            for pos in X[i].tfidf:
                centroid[pos] = X[i].tfidf[pos]
            tmp = Cluster()
            tmp.set_centroid(centroid)
            self.list_clusters.append(tmp)

    
    def init_centroid_v1(self, X):
        '''
        init centroid with K-means++ algorithm
        '''
        def add_centroid(idx):
            centroid = np.zeros(self.dim)
            for pos in X[idx].tfidf:
                centroid[pos] = X[idx].tfidf[pos]
            new_cluster = Cluster()
            new_cluster.set_centroid(centroid)
            self.list_clusters.append(new_cluster)

        def get_idx_max(list_dist):
            idx=np.argpartition(list_dist,-100)[-100:]
            ans=idx[np.argmin(list_dist[idx])]
            return ans 

        # init one centroid
        add_centroid(random.randint(1, self.n_doc))
        # min distance between point x and clusters
        dist_min_between_x_centroid=np.array([sys.maxsize]*self.n_doc,dtype=float)
        

        for k in range(self.k_cluster-1):
            for i in range(self.n_doc):
                d=self.dist_between_x_y(X[i],self.list_clusters[-1].centroid)
                dist_min_between_x_centroid[i]=min(dist_min_between_x_centroid[i],d)
            #idx_new_centroid=np.argmax(dist_min_between_x_centroid)

            # get index k-th largest element
            idx_new_centroid=get_idx_max(dist_min_between_x_centroid)
            add_centroid(idx_new_centroid)


    def update_centroid(self):
        max_simirality = -1
        for clr in self.new_clusters:
            new_centroid = np.zeros(self.dim)
            for pnt in clr.points:
                for pos in pnt.tfidf:
                    new_centroid[pos] += pnt.tfidf[pos]
            new_centroid = new_centroid/len(clr.points)
            new_centroid=new_centroid/np.linalg.norm(new_centroid)
            clr.set_centroid(new_centroid)
        

    def asign_cluster(self, X):
        self.new_clusters.clear()
        for i in range(self.k_cluster):
            tmp=Cluster()
            self.new_clusters.append(tmp)

        for x in X:
            dis_min = sys.maxsize
            asign_clr = -1
            for i in range(self.k_cluster):
                dis = self.dist_between_x_y(x,self.list_clusters[i].centroid)
                if(dis <dis_min):
                    asign_clr = i
                    dis_min = dis
            self.new_clusters[asign_clr].add_point(x)

    
    def check_stop(self, label_criteria):
        count_labels_change = self.n_doc
        for i in range(self.k_cluster):
            labels_unchange =[ label for label in self.list_clusters[i].points
                              if label in self.new_clusters[i].points ]
            count_labels_change -= len(labels_unchange)
        if(count_labels_change <label_criteria):
            return True

        return False
    
    
    def fit(self, X,k_cluster):
        def backup():
            self.list_clusters.clear()
            self.list_clusters= [ clr for clr in self.new_clusters]

        def run_batch(X_batch):
            for i in range(100):
                self.asign_cluster(X_batch)
                self.update_centroid()
                if(self.check_stop(10)):
                    self.it=i
                    backup()
                    break
                backup()

        
        self.n_doc = len(X)
        self.k_cluster = k_cluster
        self.init_centroid_v1(X)

        
        #rd =random.sample(range(self.n_doc),2000)
        #run_batch(X[rd])

        # 
        run_batch(X)

    def purity(self, X_test):
        self.asign_cluster(X_test)
        n_test = len(X_test)
        count_predict_true = 0
        for clr in self.new_clusters:
            pre = [0]*self.k_cluster
            for x in clr.points:
                pre[int(x.label)] += 1
            count_predict_true += np.max(pre)
        return str(count_predict_true)+'/'+str(n_test)


In [24]:
for i in range(5):
  t=time.time()
  model= Kmeans()
  model.fit(X_train,20)
  print("time train =",time.time()-t,'; iter =',model.it,end='; ')
  pre_train=model.purity(X_train)
  print('pre train =', ans_train,end=';')
  pre_test=model.purity(X_test)
  print('pre test= ',ans_test)

time train = 250.70914030075073 ; iter = 22; pre train = 4783/11314;pre test=  3195/7532
time train = 284.87055110931396 ; iter = 25; pre train = 6492/11314;pre test=  4077/7532
time train = 210.4250364303589 ; iter = 18; pre train = 6463/11314;pre test=  4056/7532
time train = 246.3372483253479 ; iter = 22; pre train = 5027/11314;pre test=  3169/7532
time train = 217.35833287239075 ; iter = 19; pre train = 5765/11314;pre test=  3689/7532


In [7]:
for i in range(5):
  t=time.time()
  model= Kmeans()
  model.fit(X_train,20)
  print("time train =",time.time()-t,'; iter =',model.it,end='; ')
  pre_train=model.purity(X_train)
  print('pre train =', ans_train,end=';')
  pre_test=model.purity(X_test)
  print('pre test= ',ans_test)

time train = 256.1935074329376 ; iter = 22; pre train = 5512/11314;pre test=  3667/7532
time train = 263.48727655410767 ; iter = 22; pre train = 6015/11314;pre test=  3790/7532
time train = 250.06746363639832 ; iter = 21; pre train = 5312/11314;pre test=  3362/7532
time train = 258.9289605617523 ; iter = 22; pre train = 5277/11314;pre test=  3357/7532
time train = 298.2611417770386 ; iter = 25; pre train = 5964/11314;pre test=  3788/7532


In [3]:
for i in range(10):
  t=time.time()
  model= Kmeans()
  model.fit(X_train,20)
  print("time train =",time.time()-t,'; iter =',model.it,end='; ')
  pre_train=model.purity(X_train)
  print('pre train =', ans_train,end=';')
  pre_test=model.purity(X_test)
  print('pre test= ',ans_test)

time train = 273.7457227706909 ; iter = 23; pre train = 5206/11314;pre test=  3133/7532
time train = 288.47877955436707 ; iter = 24; pre train = 5187/11314;pre test=  3326/7532
time train = 265.8109276294708 ; iter = 22; pre train = 5370/11314;pre test=  3489/7532
time train = 339.2499849796295 ; iter = 28; pre train = 6409/11314;pre test=  4040/7532
time train = 265.2014272212982 ; iter = 22; pre train = 6360/11314;pre test=  3928/7532
time train = 242.74801969528198 ; iter = 20; pre train = 4806/11314;pre test=  3090/7532
time train = 296.57629466056824 ; iter = 24; pre train = 6408/11314;pre test=  4024/7532
time train = 190.77582550048828 ; iter = 15; pre train = 5279/11314;pre test=  3251/7532
time train = 246.42981910705566 ; iter = 20; pre train = 4863/11314;pre test=  3232/7532
time train = 323.4999577999115 ; iter = 27; pre train = 5481/11314;pre test=  3340/7532


In [0]:
for i in range(10):
  t=time.time()
  model= Kmeans()
  model.fit(X_train,20)
  print("time train =",time.time()-t,'; iter =',model.it,end='; ')
  pre_train=model.purity(X_train)
  print('pre train =', ans_train,end=';')
  pre_test=model.purity(X_test)
  print('pre test= ',ans_test)

time train = 250.67050862312317 ; iter = 21; pre train = 5142/11314;pre test=  3340/7532
time train = 267.871887922287 ; iter = 23; pre train = 5194/11314;pre test=  3331/7532
time train = 276.983540058136 ; iter = 24; pre train = 5663/11314;pre test=  3596/7532
time train = 338.19768595695496 ; iter = 30; pre train = 5289/11314;pre test=  3426/7532
time train = 266.5327990055084 ; iter = 23; pre train = 5780/11314;pre test=  3682/7532
time train = 307.8704254627228 ; iter = 27; pre train = 5566/11314;pre test=  3512/7532
time train = 280.3291218280792 ; iter = 24; pre train = 5379/11314;pre test=  3586/7532
time train = 235.58061981201172 ; iter = 20; pre train = 5180/11314;pre test=  3297/7532
time train = 221.6641092300415 ; iter = 19; pre train = 5539/11314;pre test=  3742/7532
time train = 248.29695010185242 ; iter = 21; pre train = 5519/11314;pre test=  3559/7532
